In [54]:
from keras.layers import Dense, Activation
from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn import model_selection, naive_bayes, svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import pickle
pd.set_option('display.max_rows', 90)
pd.set_option('display.max_columns', 27)

In [2]:
data = pd.read_csv('data.csv',sep=',')
data
response_var = ['relevant']
predictors = list(set(list(data.columns))-set(response_var)) 

In [3]:
to_remove = np.where(data[response_var].isnull()==True)[0] #where variable to predict is NaN
data = data.drop(to_remove,axis=0)

In [4]:
columns_to_impute = data.columns[data.isnull().any()[data.columns]==True].tolist() #columns to impute
imp = SimpleImputer(strategy='mean')
data = pd.DataFrame(imp.fit_transform(data), columns=data.columns,index=data.index)

data[response_var] = data[response_var].astype(int)

In [5]:
data[predictors] = (data[predictors]-data[predictors].min())/(data[predictors].max()-data[predictors].min())
data

,quality,bits,intra_parts,skip_parts,inter_16x16_parts,inter_4x4_parts,inter_other_parts,non_zero_pixels,frame_width,frame_height,movement_level,mean,sub_mean_1,sub_mean_2,sub_mean_3,sub_mean_4,var_sub_blocks,sobel_h,sobel_v,variance,block_movement_h,block_movement_v,var_movement_h,var_movement_v,cost_1,cost_2,relevant
0,0.0,0.449172,0.00,0.00,0.25,0.090909,0.777778,0.487923,0.0,0.0,0.000874,0.003046,0.128320,0.056900,0.120360,0.160287,0.004271,0.249583,0.260537,0.043311,0.008756,0.013434,0.000055,0.000125,0.000000,0.000000,1
1,0.0,0.363655,0.00,0.50,0.00,0.090909,0.296296,0.460145,0.0,0.0,0.000874,0.002296,0.046240,0.056643,0.139796,0.135946,0.001261,0.206966,0.213153,0.023749,0.005724,0.010169,0.000032,0.000097,0.020709,0.019720,1
2,0.0,0.413121,0.00,0.00,0.00,0.000000,0.296296,0.555556,0.0,0.0,0.000874,0.002906,0.061328,0.056131,0.184563,0.187153,0.003220,0.218103,0.208956,0.027996,0.009314,0.026124,0.000008,0.000054,0.011990,0.011636,1
3,0.0,0.518340,0.00,0.00,0.00,0.545455,0.740741,0.508454,0.0,0.0,0.000874,0.003837,0.083008,0.078686,0.299267,0.163317,0.011710,0.311674,0.436763,0.139986,0.006452,0.013228,0.000133,0.000507,0.050577,0.044369,1
4,0.0,0.229092,0.00,0.00,0.25,0.090909,0.296296,0.201691,0.0,0.0,0.000874,0.001631,0.094141,0.064230,0.003903,0.043531,0.007217,0.171143,0.214952,0.086600,0.009532,0.023934,0.000114,0.000216,0.035407,0.032599,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,1.0,0.013834,0.25,0.25,0.25,0.272727,0.037037,0.003623,1.0,1.0,0.959529,0.000381,0.000000,0.000000,0.068185,0.000000,0.001867,0.020552,0.035962,0.003290,0.012807,0.071098,0.000491,0.014257,0.043937,0.040965,1
15996,1.0,0.020960,0.50,0.00,0.00,0.545455,0.074074,0.009662,1.0,1.0,0.959529,0.000973,0.064844,0.024195,0.030588,0.000000,0.003198,0.040810,0.088201,0.007544,0.011449,0.000000,0.001048,0.000000,0.098044,0.083378,1
15997,1.0,0.029554,0.25,0.25,0.00,0.000000,0.148148,0.018116,1.0,1.0,0.959529,0.001910,0.108594,0.064794,0.000000,0.080497,0.008794,0.108729,0.135351,0.025153,0.026002,0.086640,0.001139,0.011933,0.048489,0.038194,1
15998,1.0,0.070216,0.25,0.00,0.50,0.181818,0.074074,0.045894,1.0,1.0,0.959529,0.004614,0.090430,0.082274,0.067389,0.611150,0.056167,0.307822,0.515049,0.153775,0.028621,0.157903,0.000617,0.023545,0.293814,0.329225,1


In [6]:
index = data.index.tolist()
train = np.random.choice(index, np.round(len(index)*2/3).astype(int), replace=False)
test =[ele for ele in index if ele not in train]

x_train = data.loc[train][predictors]
y_train = np.ravel(data.loc[train][response_var])
x_test = data.loc[test][predictors]
y_test = np.ravel(data.loc[test][response_var])

In [7]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(x_train,y_train)
pred_NB = Naive.predict(x_test)

confusion_matrix(pred_NB,y_test)

array([[  33,   38],
       [ 879, 4383]], dtype=int64)

In [8]:
clf=RandomForestClassifier(n_estimators=1000,criterion='gini')

clf.fit(x_train,y_train)

pred_RF = clf.predict(x_test)

confusion_matrix(pred_RF,y_test)

array([[ 207,  170],
       [ 705, 4251]], dtype=int64)

In [88]:
alpha = 0.005

def build_dqn(lr,n_output,input_dims):
    model=Sequential()
    model.add(Dense(50, activation='relu', input_dim=input_dims))
    model.add(Dense(80, activation='relu', input_dim=input_dims))
    model.add(Dense(50, activation='relu', input_dim=input_dims))
    model.add(Dense(30, activation='relu', input_dim=input_dims))
    model.add(Dense(n_output, activation='softmax'))
    model.compile(optimizer=Adam(lr=alpha),
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
    return model

dqn = build_dqn(alpha,2,26)
dqn.fit(x_train, to_categorical(y_train), epochs=500, batch_size=100,class_weight = {0: 0.85, 1: 0.15})


Epoch 1/500
10665/10665 [==============================] - 1s 55us/step - loss: 0.1625 - accuracy: 0.6092
Epoch 2/500
10665/10665 [==============================] - 0s 23us/step - loss: 0.1552 - accuracy: 0.6237
Epoch 3/500
10665/10665 [==============================] - 0s 22us/step - loss: 0.1535 - accuracy: 0.6110
Epoch 4/500
10665/10665 [==============================] - 0s 22us/step - loss: 0.1532 - accuracy: 0.6303
Epoch 5/500
10665/10665 [==============================] - 0s 24us/step - loss: 0.1514 - accuracy: 0.6321
Epoch 6/500
10665/10665 [==============================] - 0s 22us/step - loss: 0.1519 - accuracy: 0.6444
Epoch 7/500
10665/10665 [==============================] - 0s 23us/step - loss: 0.1506 - accuracy: 0.6326
Epoch 8/500
10665/10665 [==============================] - 0s 23us/step - loss: 0.1505 - accuracy: 0.6405
Epoch 9/500
10665/10665 [==============================] - 0s 23us/step - loss: 0.1490 - accuracy: 0.6453
Epoch 10/500
10665/10665 [====================

In [89]:
pred_NN = np.argmax((dqn.predict(x_train)), axis = 1)
confusion_matrix(pred_NN,y_train)
confusion_matrix(pred_NN,y_train)

array([[1898, 1461],
       [  32, 7274]], dtype=int64)

In [90]:
pred_NN = np.argmax((dqn.predict(x_test)), axis = 1)
confusion_matrix(pred_NN,y_test)
confusion_matrix(pred_NN,y_test)

array([[ 577, 1154],
       [ 335, 3267]], dtype=int64)

In [78]:
model = dqn
pred_train= model.predict(x_train)
scores = model.evaluate(x_train, to_categorical(y_train), verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))   
 
pred_test= model.predict(x_test)
scores2 = model.evaluate(x_test, to_categorical(y_test), verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))    

Accuracy on training data: 0.745147705078125% 
 Error on training data: 0.254852294921875
Accuracy on test data: 0.658916175365448% 
 Error on test data: 0.341083824634552


In [55]:
with open('model2.pickle', 'wb') as f:
    pickle.dump(dqn, f)